# Exchange Rate Data

This script gathers currency exchange rate data using API calls to the AlphaVantage Monthly Foreign Exchange (FX_MONTHLY) dataset.  The data is saved in a dataframe, and is cleaned to preserve the greatest number of common datapoints across currency. The data is written to a CSV file in the Output directory.

In [7]:
# set environment
import pycountry
import requests
import json
import pandas as pd



# pull constants from config file
from config import base_country
from config import countries
from config import xchg_filename

# retrieve API key
from api_keys import akavantage


# create empty lists for our dataframe data
country_name = []
currency_name = []
currency_code = []
year = []
close_rate = []


In [8]:

# all exchange rates are in reference to the base currency
base_currency = base_country["currency_code"]

# loop through our list of currencies and pull data from AlphaVantage
for country in countries:

    # pull currency code associated with next country in list
    from_currency = countries[country]["currency_code"]
    
    # build URL for AlphaVantage
    url = f"https://www.alphavantage.co/query?function=FX_MONTHLY&from_symbol={from_currency}&to_symbol={base_currency}&apikey={akavantage}"

    # request data
    try:
        response = requests.get(url).json()
    
        # loop through datapoints and populate data lists
        for record_date in response["Time Series FX (Monthly)"]:
            
            # pull date field apart
            date_list = record_date.split("-")
            
            # check for year-end data (month = 12) and save to data lists
            if (date_list[1] == "12"):
                currency_code.append(from_currency)
                country_name.append(country)
                currency_name.append(countries[country]["currency_name"])
                year.append(date_list[0])
                close_rate.append(response["Time Series FX (Monthly)"][record_date]["4. close"])

    except:
        # catch error but keep going
        print(f"missing data: {from_currency}")

# retrieval takes time, so print when complete
print("Retrieval of currency exchange data complete.")
        
            


Retrieval of currency exchange data complete.


In [9]:
# build dataframe
exchange_data = pd.DataFrame({"Country": country_name, "Currency Code": currency_code, "Currency": currency_name, "Year": year, "Close Rate": close_rate})
exchange_data.head()

,Country,Currency Code,Currency,Year,Close Rate
0,European Union,EUR,Euro,2020,1.2214
1,European Union,EUR,Euro,2019,1.1213
2,European Union,EUR,Euro,2018,1.1466
3,European Union,EUR,Euro,2017,1.1994
4,European Union,EUR,Euro,2016,1.0516


In [10]:
# remove datapoints before 2014 and after 2020
analysis_data = exchange_data.loc[(exchange_data["Year"] >= "2014") & (exchange_data["Year"] < "2020")]

# just a check for consistent datapoints across currencies
analysis_data.groupby("Currency Code").count()

,Country,Currency,Year,Close Rate
Currency Code,,,,
AUD,6,6,6,6
BRL,6,6,6,6
BYN,4,4,4,4
CAD,6,6,6,6
CHF,6,6,6,6
EUR,6,6,6,6
GBP,6,6,6,6
HKD,6,6,6,6
INR,6,6,6,6


In [12]:
analysis_data = analysis_data.drop(analysis_data[analysis_data["Currency Code"] == "BYN"].index)
analysis_data.groupby("Currency Code").count()

,Country,Currency,Year,Close Rate
Currency Code,,,,
AUD,6,6,6,6
BRL,6,6,6,6
CAD,6,6,6,6
CHF,6,6,6,6
EUR,6,6,6,6
GBP,6,6,6,6
HKD,6,6,6,6
INR,6,6,6,6
JPY,6,6,6,6


In [13]:
# sort dataframe by currency and year
sorted_analysis = analysis_data.sort_values(by=["Currency Code", "Year"])

# write to CSV for plotting
sorted_analysis.to_csv(xchg_filename, index=False)
sorted_analysis

,Country,Currency Code,Currency,Year,Close Rate
53,Australia,AUD,Australian Dollar,2014,0.8165
52,Australia,AUD,Australian Dollar,2015,0.7289
51,Australia,AUD,Australian Dollar,2016,0.7201
50,Australia,AUD,Australian Dollar,2017,0.7808
49,Australia,AUD,Australian Dollar,2018,0.7048
...,...,...,...,...,...
154,Turkey,TRY,Turkish Lira,2015,0.3423
153,Turkey,TRY,Turkish Lira,2016,0.2830
152,Turkey,TRY,Turkish Lira,2017,0.2633
151,Turkey,TRY,Turkish Lira,2018,0.1884
